# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Requirements

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project._

* Code clarity: make sure the code conforms to:
    * [ ] [PEP 8](https://peps.python.org/pep-0008/) - You might find [this resource](https://realpython.com/python-pep8/) helpful as well as [this](https://github.com/dnanhkhoa/nb_black) or [this](https://jupyterlab-code-formatter.readthedocs.io/en/latest/) tool
    * [ ] [PEP 257](https://peps.python.org/pep-0257/)
    * [ ] Break each task down into logical functions
* The following files are submitted for the project (see the project's GDoc for more details):
    * [ ] `README.md`
    * [ ] `requirements.txt`
    * [ ] `.gitignore`
    * [ ] `schema.sql`
    * [ ] 6 query files (using the `.sql` extension), appropriately named for the purpose of the query
    * [x] Jupyter Notebook containing the project (this file!)
* [x] You can edit this cell and add a `x` inside the `[ ]` like this task to denote a completed task

## Project Setup

In [2]:
# !pip install matplotlib
# !pip install  fastparquet==0.7.1
# !pip install geopandas

In [4]:
# all import statements needed for the project, for example:

import math
import requests
import re
import bs4
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import pandas as pd
import requests
import sqlalchemy as db
import geopandas as gpd

In [5]:
# any general notebook setup, like log formatting


In [1]:
# any constants you might need, for example:

TAXI_URL = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

# add other constants to refer to any local data, e.g. uber & weather
UBER_CSV = "uber_rides_sample.csv"
weather_2009_CSV="2009_weather.csv"
weather_2010_CSV="2010_weather.csv"
weather_2011_CSV="2011_weather.csv"
weather_2012_CSV="2012_weather.csv"
weather_2013_CSV="2013_weather.csv"
weather_2014_CSV="2014_weather.csv"


NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

## Part 1: Data Preprocessing

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Define a function that calculates the distance between two coordinates in kilometers that **only uses the `math` module** from the standard library.
* [ ] Taxi data:
    * [ ] Use the `re` module, and the packages `requests`, BeautifulSoup (`bs4`), and (optionally) `pandas` to programmatically download the required CSV files & load into memory.
    * You may need to do this one file at a time - download, clean, sample. You can cache the sampling by saving it as a CSV file (and thereby freeing up memory on your computer) before moving onto the next file. 
* [ ] Weather & Uber data:
    * [ ] Download the data manually in the link provided in the project doc.
* [ ] All data:
    * [ ] Load the data using `pandas`
    * [ ] Clean the data, including:
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * (Taxi & Uber data) Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
    * [ ] (Taxi data) Sample the data so that you have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
* [ ] Weather data:
    * [ ] Split into two `pandas` DataFrames: one for required hourly data, and one for the required daily daya.
    * [ ] You may find that the weather data you need later on does not exist at the frequency needed (daily vs hourly). You may calculate/generate samples from one to populate the other. Just document what you’re doing so we can follow along. 

### Calculating distance
_**TODO:** Write some prose that tells the reader what you're about to do here._

In [16]:
def calculate_distance(from_coord, to_coord):
    #return the euclidean distance between start and end coordinate
    from math import sin, cos, sqrt, atan2, radians
    R = 6373.0 #approximate radius of earth in km
    lon1=radians(from_coord[0])
    lat1=radians(from_coord[1])
    lon2=radians(to_coord[0])
    lat2=radians(to_coord[1])
    dlon=lon2-lon1
    dlat=lat2-lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    b = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance=R*b
    return distance

In [ ]:
def get_lon_lat(dataframe):
    zone_file=gpd.read_file("taxi_zones")
    zone_file['geometry']=zone_file['geometry'].to_crs(4326)
    zone_file["lon"]=zone_file['geometry'].centroid.x
    zone_file["lat"]=zone_file['geometry'].centroid.y
    pick=zone_file[["LocationID","lon","lat"]]
    pick.columns=["PULocationID","pickup_longitude","pickup_latitude"]
    drop=zone_file[["LocationID","lon","lat"]]
    drop.columns=["DOLocationID","dropoff_longitude","dropoff_latitude"]
    #merge pick and drop to dataframe
    #merge pick 
    dataframe=pd.merge(left=dataframe,right=pick)
    #merge drop
    dataframe=pd.merge(left=dataframe,right=drop)
    return dataframe

In [17]:
def add_distance_column(dataframe):
    dataframe["distance"]=dataframe.apply(lambda x: calculate_distance(x["pickup_longitude","pickup_latitude"],
    x["dropoff_longitude","dropoff_latitude"]),axis=1)
    return dataframe 

In [27]:
zone_file=gpd.read_file("taxi_zones")
zone_file['geometry']=zone_file['geometry'].to_crs(4326)
zone_file["lon"]=zone_file['geometry'].centroid.x
zone_file["lat"]=zone_file['geometry'].centroid.y
zone_file[["LocationID","lon","lat"]]

/var/folders/kh/jc_vc4mj42s708qkj0c26b6w0000gn/T/ipykernel_16211/1654112693.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  zone_file["lon"]=zone_file['geometry'].centroid.x
/var/folders/kh/jc_vc4mj42s708qkj0c26b6w0000gn/T/ipykernel_16211/1654112693.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  zone_file["lat"]=zone_file['geometry'].centroid.y


,LocationID,lon,lat
0,1,-74.174000,40.691831
1,2,-73.831299,40.616745
2,3,-73.847422,40.864474
3,4,-73.976968,40.723752
4,5,-74.188484,40.552659
...,...,...,...
258,259,-73.852215,40.897932
259,260,-73.906306,40.744235
260,261,-74.013023,40.709139
261,262,-73.946510,40.775932


From above table, we know that loction ID is valid between 1 and 263. Thus, we can use it to clean our taxi dataset.

### Processing Taxi Data


Before we processing taxi data, let's have a explore one of the taxi dataset. So that we know how to clean the dataset. First, we write a function called find_taxi_Parquet_urls, which will return urls for Parquet dataset from January 2009 through June 2015.

In [24]:
def find_taxi_Parquet_urls():
    url=TAXI_URL #url for taxi data page is set above
    url_list=list()
    response=requests.get(url)
    if response.status_code!=200:
        print("Failure: fail to request data")
        return result
    try:
        result_page=BeautifulSoup(response.content,'lxml')
        result_tags=result_page.find_all("a",{"title":"Yellow Taxi Trip Records"})
        for element in result_tags:
            newurl=element.get('href')
            url_list.append(newurl)
        result=url_list[80:86]
        result.extend(url_list[92:]) #only include dataset link from January 2009 through June 2015 
    except:
        print("Failure: scraping fail")
    return result



In [32]:
result=find_taxi_Parquet_urls()
result[0]

'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-06.parquet'

Next, we explore one of the taxi dataset to deside how to process the data. Here we choose 2015 June. 

In [33]:
#choose 2015 June. 
data_url=result[0]
df=pd.read_parquet(data_url,engine='fastparquet')

In [31]:
df.head()

,vendor_name,Trip_Pickup_DateTime,Trip_Dropoff_DateTime,Passenger_Count,Trip_Distance,Start_Lon,Start_Lat,Rate_Code,store_and_forward,End_Lon,End_Lat,Payment_Type,Fare_Amt,surcharge,mta_tax,Tip_Amt,Tolls_Amt,Total_Amt
0,VTS,2009-12-17 07:35:00,2009-12-17 07:40:00,1,0.11,-73.987928,40.737885,NaN,NaN,-73.990335,40.748450,Credit,4.9,0.0,0.5,1.0,0.0,6.4
1,VTS,2009-12-21 14:19:00,2009-12-21 14:24:00,1,1.07,-73.956008,40.779558,NaN,NaN,-73.967303,40.787833,CASH,4.9,0.0,0.5,0.0,0.0,5.4
2,VTS,2009-12-18 03:09:00,2009-12-18 03:34:00,1,8.98,-73.955745,40.689503,NaN,NaN,-73.937730,40.737463,CASH,23.7,0.5,0.5,0.0,0.0,24.7
3,VTS,2009-12-14 21:24:00,2009-12-14 21:33:00,2,1.66,-73.983985,40.754645,NaN,NaN,-73.986195,40.737610,Credit,6.9,0.5,0.5,3.0,0.0,10.9
4,VTS,2009-12-18 08:17:00,2009-12-18 08:29:00,1,1.55,-73.959132,40.769265,NaN,NaN,-73.976267,40.760615,CASH,7.7,0.0,0.5,0.0,0.0,8.2


In [18]:
a=add_distance_column(df)

/var/folders/kh/jc_vc4mj42s708qkj0c26b6w0000gn/T/ipykernel_16211/325126572.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  zone_file["lon"]=zone_file['geometry'].centroid.x
/var/folders/kh/jc_vc4mj42s708qkj0c26b6w0000gn/T/ipykernel_16211/325126572.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  zone_file["lat"]=zone_file['geometry'].centroid.y


In [19]:
a["distance"]=a.apply(lambda x:calculate_distance([x["Pick_lon"],x["Pick_lat"]], [x["Drop_lon"],x["Drop_lat"]]),axis=1)

In [20]:
a

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,Pick_lon,Pick_lat,Drop_lon,Drop_lat,distance
0,1,2015-06-01 00:04:04,2015-06-01 00:13:02,1,1.10,1,N,79,148,3,...,0.0,0.3,8.80,NaN,NaN,-73.985937,40.727620,-73.990896,40.718938,1.052271
1,1,2015-06-01 00:52:54,2015-06-01 00:56:33,2,0.80,1,N,79,148,1,...,0.0,0.3,7.56,NaN,NaN,-73.985937,40.727620,-73.990896,40.718938,1.052271
2,2,2015-06-01 00:00:42,2015-06-01 00:05:20,3,1.14,1,N,79,148,2,...,0.0,0.3,6.80,NaN,NaN,-73.985937,40.727620,-73.990896,40.718938,1.052271
3,1,2015-06-01 00:28:04,2015-06-01 00:30:41,1,0.50,1,N,79,148,1,...,0.0,0.3,6.35,NaN,NaN,-73.985937,40.727620,-73.990896,40.718938,1.052271
4,2,2015-06-01 00:36:29,2015-06-01 00:37:23,1,0.19,1,N,79,148,1,...,0.0,0.3,5.38,NaN,NaN,-73.985937,40.727620,-73.990896,40.718938,1.052271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12098116,1,2015-06-16 08:27:15,2015-06-16 08:35:49,1,1.00,1,N,199,199,1,...,0.0,0.3,10.14,NaN,NaN,-73.882658,40.791133,-73.882658,40.791133,0.000000
12098117,1,2015-06-16 08:40:36,2015-06-16 08:56:38,1,2.80,1,N,199,199,1,...,0.0,0.3,16.56,NaN,NaN,-73.882658,40.791133,-73.882658,40.791133,0.000000
12098118,1,2015-06-16 08:59:13,2015-06-16 09:14:07,1,1.70,1,N,199,199,2,...,0.0,0.3,11.80,NaN,NaN,-73.882658,40.791133,-73.882658,40.791133,0.000000
12098119,1,2015-06-16 09:15:56,2015-06-16 09:42:44,2,6.10,1,N,199,199,1,...,0.0,0.3,29.76,NaN,NaN,-73.882658,40.791133,-73.882658,40.791133,0.000000


In [26]:
a["PULocationID"].unique()

array([ 79,  87, 164, 163, 132, 237, 236, 229, 233, 234, 142, 243, 189,
       148, 137, 249, 190,  62, 138, 230, 255,  37,  48, 170,  50,  74,
       114, 140, 141, 161, 186, 100, 144, 238, 211, 246, 239, 116, 162,
       261,  75,  90, 143, 152, 244, 226,   7,  41,  68, 181, 107,  43,
       113, 231, 125, 158,  42, 151, 166,  24,  88,  45, 263,   4, 146,
        13, 112, 256, 179,  25, 168,  97,  82, 145, 247,  40,  17,  49,
       223, 127, 262, 219,  65,  33, 232,  92,  61, 260,  80, 129,  10,
        70, 197, 224, 130,  95,  36, 134, 106,  52, 225,  63, 217, 188,
        66, 209, 133,  56, 194, 215,  83,  28, 228, 257,  93, 157,  89,
        12,  85, 193,  54,  26,  14, 195, 124,  76, 202, 216, 119, 123,
       191,  71,  34, 242, 165, 196, 136, 173,   8,  15, 135, 227, 198,
        39, 207, 160, 102, 235, 241, 131, 121,  53, 240, 171, 253, 192,
       108,  96,  98,  73,   2, 177, 178,  69, 159,  94,  35, 150,  91,
       180, 185, 208, 220, 175, 258, 126, 218,  55, 147, 200,  2

In [32]:
a.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                   int64
trip_distance                   float64
RatecodeID                        int64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
Pick_lon                        float64
Pick_lat                        float64
Drop_lon                        float64
Drop_lat                        float64
distance                        float64
dtype: object

In [28]:
def get_and_clean_month_taxi_data(url):
    #get month Taxi data
    df=pd.read_parquet(url,engine='fastparquet')
    #clean data
    #select column
    df=df[["tpep_pickup_datetime","trip_distance","PULocationID","DOLocationID","tip_amount"]]
    #remove any row with nan value
    df.dropna()
    #remove invalid locationID：location ID should between 1 and 263, and should be numeric
    df[df[['PULocationID']].apply(lambda x: x[0].isdigit(), axis=1)]
    df[df[['DOLocationID']].apply(lambda x: x[0].isdigit(), axis=1)]
    df=df.loc[df["PULocationID"]>=1 and df["PULocationID"]<=263 and df["DOLocationID"]>=1 and df["DOLocationID"]<=263]
    #add longitude and latitude based on location ID
    df=get_lon_lat(df)
    #appropriate column type

    #remove outside the latitude/longitude coordinate box
    df=df.loc[df["pickup_longitude"]<=-73.717047 and df["pickup_longitude"]>=-74.242330 
    and df["pickup_latitude"]<=40.908524 and df["pickup_latitude"]>=40.560445
    and df["dropoff_longitude"]<=-73.717047 and df["dropoff_longitude"]>=-74.242330 
    and df["dropoff_latitude"]<=40.908524  and df["dropoff_latitude"]>=40.560445]

    #normalize and usng appropriate coumn type

    return df


    


    

In [ ]:
def get_and_clean_taxi_data():
    all_taxi_dataframes = []
    
    all_csv_urls = find_taxi_Parquet_urls()
    for csv_url in all_csv_urls:
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_month_taxi_data(csv_url)
        #add distance column 
        dataframe=add_distance_column(dataframe)
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function
        
        all_taxi_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.contact(all_taxi_dataframes)
    return taxi_data

### Processing Uber Data


In [31]:
uber=pd.read_csv("uber_rides_sample.csv")
uber.head()

,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


In [ ]:
def load_and_clean_uber_data(csv_file):
    df=pd.read_csv(csv_file)
    #clean data
    

In [ ]:
def get_uber_data():
    uber_dataframe = load_and_clean_uber_data(UBER_DATA)
    add_distance_column(uber_dataframe)
    return uber_dataframe

### Processing Weather Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [ ]:
def clean_month_weather_data_hourly(csv_file):
    raise NotImplemented()

In [ ]:
def clean_month_weather_data_daily(csv_file):
    raise NotImplemented()

In [ ]:
def load_and_clean_weather_data():
    hourly_dataframes = []
    daily_dataframes = []
    
    # add some way to find all weather CSV files
    # or just add the name/paths manually
    weather_csv_files = ["TODO"]
    
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

### Process All Data

_This is where you can actually execute all the required functions._

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [ ]:
taxi_data = get_and_clean_taxi_data()
uber_data = get_uber_data()
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

## Part 2: Storing Cleaned Data

_Write some prose that tells the reader what you're about to do here._

In [ ]:
engine = db.create_engine(DATABASE_URL)

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
TODO
"""

DAILY_WEATHER_SCHEMA = """
TODO
"""

TAXI_TRIPS_SCHEMA = """
TODO
"""

UBER_TRIPS_SCHEMA = """
TODO
"""

In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [ ]:
# create the tables with the schema files
with engine.connect() as connection:
    pass

### Add Data to Database

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [ ]:
def write_dataframes_to_table(table_to_df_dict):
    raise NotImplemented()

In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
}

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.
* [ ] For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.
* [ ] What is the 95% percentile of distance traveled for all hired trips during July 2013?
* [ ] What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?
* [ ] Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?
* [ ] During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?

In [ ]:
def write_query_to_file(query, outfile):
    raise NotImplemented()

### Query N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each query_

In [ ]:
QUERY_N = """
TODO
"""

In [ ]:
engine.execute(QUERY_N).fetchall()

In [ ]:
write_query_to_file(QUERY_N, "some_descriptive_name.sql")

## Part 4: Visualizing the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Create an appropriate visualization for the first query/question in part 3
* [ ] Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month). Include the 90% confidence interval around the mean in the visualization
* [ ] Define three lat/long coordinate boxes around the three major New York airports: LGA, JFK, and EWR (you can use bboxfinder to help). Create a visualization that compares what day of the week was most popular for drop offs for each airport.
* [ ] Create a heatmap of all hired trips over a map of the area. Consider using KeplerGL or another library that helps generate geospatial visualizations.
* [ ] Create a scatter plot that compares tip amount versus distance.
* [ ] Create another scatter plot that compares tip amount versus precipitation amount.

_Be sure these cells are executed so that the visualizations are rendered when the notebook is submitted._

### Visualization N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each visualization._

_The example below makes use of the `matplotlib` library. There are other libraries, including `pandas` built-in plotting library, kepler for geospatial data representation, `seaborn`, and others._

In [ ]:
# use a more descriptive name for your function
def plot_visual_n(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_n():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplemented()

In [ ]:
some_dataframe = get_data_for_visual_n()
plot_visual_n(some_dataframe)